In [1]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import mysql.connector
import psycopg2
import ftplib
from ftplib import FTP_TLS
import ssl
import pandas as pd
import datetime
import numpy as np
import datetime
import shutil

In [3]:
localFilePath = '/home/elliott/development/files/'
changeFileName = 'change.csv'
positionsFileName = 'positions.csv'

d = datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=1)
d2 = d - datetime.timedelta(days=1)
d3 = d2 - datetime.timedelta(days=1)

# Open the local files
localFile = open(os.path.join(localFilePath, changeFileName), 'wb')
localFileP = open(os.path.join(localFilePath, positionsFileName), 'wb')

# Open the local change file and write its content to localFile
try:
    with open('Scalp_Trade_EXT982_' + d2.strftime('%Y%m%d') + ".csv", 'rb') as changeFile:
        localFile.write(changeFile.read())
except FileNotFoundError as e:
    print(f"Error: {changeFileName} not found in {localFilePath}")
    print(e)

# Open the local positions file and write its content to localFileP
try:
    with open('Scalp_Trade_EXT982_' + d2.strftime('%Y%m%d') + ".csv", 'rb') as positionsFile:
        localFileP.write(positionsFile.read())
except FileNotFoundError as e:
    print(f"Error: {positionsFileName} not found in {localFilePath}")
    print(e)

# Close the local files
localFile.close()
localFileP.close()

In [5]:
sterling = pd.read_csv('/home/elliott/development/files/change.csv', sep=',', header=0)
positions = pd.read_csv('/home/elliott/development/files/positions.csv', sep=',', header=0)
fast = pd.read_csv('/home/elliott/development/files/fast.csv', sep=',', header=None, lineterminator='\n')

/tmp/ipykernel_79579/407987990.py:3: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,26,28,33,35,37,38,39,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  fast = pd.read_csv('/home/elliott/development/files/fast.csv', sep=',', header=None, lineterminator='\n')


In [6]:
positionDict = {}
symbolsList = []
for x in range(len(positions)):
    sym = positions[7][x].replace(" ", "")
    trader = positions[4][x]
    positionDict.setdefault(trader[-4:], {}).setdefault(sym, positions[10][x])
    if sym not in symbolsList:
        symbolsList.append(sym)

print(positionDict.keys())
fastLength = fast.shape[0]
fastTimes = fast.iloc[:, 31]
spoofs = pd.DataFrame(np.zeros((3500, 16)), columns=["Date", "Time", "Time Entered", "Side", "Symbol",
                                                     "Position", "Quantity", "Maturity", "Strike Price",
                                                     "Order Price", "Open Close", "Order Type", "OrderID",
                                                     "Status", "Exchange", "Account"])
spIt = 0
start = datetime.datetime.now()
for i in range(fastLength - 1):
    if fast.iat[i, 17] == '0' or fast.iat[i, 17] == '(null)':
        continue
    temp = i + 1
    while (fast.iat[temp, 22] == '(null)'):
        temp += 1
    symbol = fast.iat[i, 33]
    account = fast.iat[i, 1]
    tradr = account[3:7] if str(account[2]).isalpha() else account[2:6]
    amt = int(fast.iat[i, 17])
    spacedSymbol = symbol
    try:
        t1 = datetime.datetime.strptime(fastTimes.iat[i], '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        t1 = datetime.datetime.strptime(fastTimes.iat[i], '%Y-%m-%d %H:%M:%S')
    try:
        t0 = datetime.datetime.strptime(fastTimes.iat[temp], '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        t0 = datetime.datetime.strptime(fastTimes.iat[temp], '%Y-%m-%d %H:%M:%S')

    t2 = t0
    if fast.iat[i, 32] == '2':  # side 2 = sell?
        side = 'S'
    else:
        side = 'B'
    if fast.iat[i, 68] != '(null)':
        spacedSymbol = str(symbol) + str(fast.iat[i, 66]) + str(fast.iat[i, 71]) + (
            'C' if fast.iat[i, 67] == 0 else 'P') + "{:.3f}".format(float(fast.iat[i, 68]))
        curPosition = positionDict.setdefault(tradr, {}).setdefault(spacedSymbol, 0)
    else:
        curPosition = positionDict.setdefault(tradr, {}).setdefault(symbol, 0)
    # spoofString = t1.strftime('%m/%d/%Y %H:%M:%S.%f') + "\n" + fast[7][i] + "   " +  sterling['Log'][i] +  "    " + fast[1][i]+ "\n"
    spoofFlag = False
    tempDict = {}
    while (t2 - t1) < datetime.timedelta(seconds=2.0):
        symbolT = fast.iat[temp, 33]
        accountT = fast.iat[temp, 1]
        tradrT = accountT[3:7] if str(accountT[2]).isalpha() else accountT[2:6]
        indxT = symbolT.find(' ')
        amtT = int(fast.iat[temp, 22])
        spacedSymbolT = symbolT
        if fast.iat[temp, 32] == '2':
            sideT = 'S'
        else:
            sideT = 'B'
        if fast.iat[temp, 68] != '(null)':
            spacedSymbolT = str(symbol) + str(fast.iat[temp, 66]) + str(fast.iat[temp, 71]) + (
                'C' if fast.iat[temp, 67] == 0 else 'P') + "{:.3f}".format(float(fast.iat[temp, 68]))
            tempos = positionDict.setdefault(tradrT, {}).setdefault(spacedSymbolT, 0)
            tempPosition = tempDict.setdefault(tradrT, {}).setdefault(spacedSymbolT, tempos)
        else:
            tempos = positionDict.setdefault(tradrT, {}).setdefault(spacedSymbolT, 0)
            tempPosition = tempDict.setdefault(tradrT, {}).setdefault(symbolT, tempos)
        '''
        if(temp -i) > 10:
            break

        if(symbol == symbolT and side != sideT and fast.iat[temp,68] == fast.iat[i,68] and fast.iat[temp,27] != fast.iat[i,27] and fast.iat[temp,23] == 4 and fast.iat[temp,67] == fast.iat[i,67]):
            print("XD")
        '''
        if fast.iat[i, 27] != fast.iat[temp, 27] and fast.iat[temp, 23] == 4 and \
                fast.iat[i, 68] == fast.iat[temp, 68] and fast.iat[i, 33] == \
                fast.iat[temp, 33] and (side != sideT) and (
                fast.iat[i, 67] == fast.iat[temp, 67]) and (
                (str(fast.iat[i, 66]) + str(fast.iat[i, 71])) == (str(fast.iat[temp, 66]) + str(fast.iat[temp, 71]))):

            timeEntered = findEntryF(fast.iat[temp, 21], i)
            if not spoofFlag and timeEntered != "Not Found":
                spoofFlag = True
                spoofs.loc[spIt, "Date"] = d.date()
                spoofs.loc[spIt, "Time"] = t1.strftime('%H:%M:%S.%f')
                spoofs.loc[spIt, "Time Entered"] = "N/A"
                spoofs.loc[spIt, "Side"] = side
                spoofs.loc[spIt, "Symbol"] = spacedSymbol
                spoofs.loc[spIt, "Quantity"] = amt
                spoofs.loc[spIt, "Maturity"] = str(fast.iat[i, 66]) + str(fast.iat[i, 71])
                spoofs.loc[spIt, "Strike Price"] = fast.iat[i, 68]
                spoofs.loc[spIt, "Order Price"] = fast.iat[i, 27]
                spoofs.loc[spIt, "Open Close"] = fast.iat[i, 41]
                spoofs.loc[spIt, "Order Type"] = fast.iat[i, 24]
                spoofs.loc[spIt, "Status"] = 'Filled' if fast.iat[i, 23] == 2 else 'Canceled'
                spoofs.loc[spIt, "OrderID"] = fast.iat[i, 21]
                spoofs.loc[spIt, "Exchange"] = fast.iat[i, 15]
                spoofs.loc[spIt, "Account"] = account
                spoofs.loc[spIt, "Position"] = curPosition
                iSpIt = spIt
                spIt += 1
                if fast.iat[i, 68] != '(null)':
                    tempDict.setdefault(tradr, {}).setdefault(spacedSymbol, 0)
                else:
                    tempDict.setdefault(tradr, {}).setdefault(symbol, 0)
                if side == 'S':
                    tempDict[tradr][spacedSymbol] -= amt
                else:
                    tempDict[tradr][spacedSymbol] += amt

            if timeEntered != 'Not Found':
                spoofs.loc[spIt, "Date"] = d.date()
                spoofs.loc[spIt, "Time"] = t2.strftime('%H:%M:%S.%f')
                spoofs.loc[spIt, "Time Entered"] = timeEntered
                print(timeEntered)
                spoofs.loc[spIt, "Side"] = sideT
                spoofs.loc[spIt, "Symbol"] = spacedSymbolT
                spoofs.loc[spIt, "Quantity"] = fast.iat[temp, 22]
                spoofs.loc[spIt, "Maturity"] = str(fast.iat[i, 66]) + str(fast.iat[i, 71])
                spoofs.loc[spIt, "Strike Price"] = fast.iat[temp, 68]
                spoofs.loc[spIt, "Order Price"] = fast.iat[temp, 27]
                spoofs.loc[spIt, "Order Type"] = fast.iat[temp, 24]
                spoofs.loc[spIt, "Status"] = 'Filled' if fast.iat[temp, 23] == 2 else 'Canceled'
                spoofs.loc[spIt, "OrderID"] = fast.iat[temp, 21]
                spoofs.loc[spIt, "Exchange"] = 'N/A'
                account = fast.iat[temp, 1]
                spoofs.loc[spIt, "Account"] = accountT
                spoofs.loc[spIt, "Open Close"] = fast.iat[temp, 41]
                spoofs.loc[spIt, "Position"] = tempDict[tradrT][spacedSymbolT]

                spIt += 1
        temp += 1
        if temp == (fastLength):
            break
        while fast.iat[temp, 22] == '(null)':
            temp += 1
        try:
            t2 = datetime.datetime.strptime(fastTimes[temp], '%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            t2 = datetime.datetime.strptime(fastTimes[temp], '%Y-%m-%d %H:%M:%S')

    if (side == 'S'):
        positionDict[tradr][spacedSymbol] -= amt
    else:
        positionDict[tradr][spacedSymbol] += amt

    if t0 - t1 < datetime.timedelta(seconds=2) and spoofFlag:
        spoofs.loc[spIt, "Date"] = ' '
        spoofs.loc[spIt, "Time"] = ' '
        spoofs.loc[spIt, "Time Entered"] = ' '
        spoofs.loc[spIt, "Side"] = ' '
        spoofs.loc[spIt, "Symbol"] = ' '
        spoofs.loc[spIt, "Quantity"] = ' '
        spoofs.loc[spIt, "Maturity"] = ' '
        spoofs.loc[spIt, "Strike Price"] = ' '
        spoofs.loc[spIt, "Order Price"] = ' '
        spoofs.loc[spIt, "Order Type"] = ' '
        spoofs.loc[spIt, "Status"] = ' '
        spoofs.loc[spIt, "OrderID"] = ' '
        spoofs.loc[spIt, "Exchange"] = ' '
        spoofs.loc[spIt, "Account"] = ' '
        spoofs.loc[spIt, "Open Close"] = ' '
        spoofs.loc[spIt, "Position"] = ' '

        spIt += 1
print(datetime.datetime.now() - start)
print(positionDict.keys())
spoofs = spoofs[:spIt]
spoofFile2 = "/tmp/report-files/Spoofs_Fast_" + d.strftime('%Y%m%d') + ".csv"
spoofs.to_csv(spoofFile2, date_format='%s')

KeyError: 7